<a href="https://colab.research.google.com/github/amir-10/Projet_Big_Data/blob/main/Projet_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# instalation de la bibliotheque java
! apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
# instalation de spark
! wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

In [5]:
# décompresser le dossier 
! tar xf /content/spark-3.3.1-bin-hadoop3.tgz


In [6]:
# instalation de pyspark
! pip install -q findspark
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=5cd5abd2d0b5b69a9f9cf5eec014b877e88f79a2a7d676234fb6c78e55694abe
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [7]:
import os
# definir deux variables d'environement

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"  # l'endroit de java
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3" # l'endroit de spark
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.5 pyspark-shell' 
import findspark

findspark.init("spark-3.3.1-bin-hadoop3")


from pyspark import SparkContext, SparkConf

# lancement du spark en local (en une seule machine qui est la machine virtuelle de colab dans ce cas) avec 4 processus (workers node)
configuration = SparkConf().setAppName("name").setMaster("local[4]").set('spark.jars.packages', 'org.apache.spark:spark-avro_2.11:2.4.5')
# name: c'est le nom qu'on donne a notre app (code)
sc = SparkContext(conf=configuration) # l'objet 

In [8]:
sc

<SparkContext master=local[4] appName=name>

In [9]:
# L'objet sparkSession créé pour utiliser l'API Spark SQL
from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=configuration).getOrCreate()

In [10]:
! wget -q http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz

In [11]:
# décompresser le dossier 
! tar xf /content/20news-19997.tar.gz

In [22]:
rdd1 = sc.wholeTextFiles("/content/20_newsgroups/alt.atheism")
rdd2 = sc.wholeTextFiles("/content/20_newsgroups/rec.sport.baseball")

In [23]:
x=rdd2.take(2)

In [24]:
def separateur(x):
  l=x[1].split("\n\n")
  return (x[0],l)

#separateur(x[1])

rdd1=rdd1.map(separateur)
rdd2=rdd2.map(separateur)






In [90]:
# extraction des infos de l'entete 
test=rdd1.take(1)
test

[('file:/content/20_newsgroups/alt.atheism/53079',
  ["Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:120533 alt.atheism:53079 talk.religion.misc:83551\nPath: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!uunet!mcsun!Germany.EU.net!thoth.mchp.sni.de!horus.ap.mchp.sni.de!D012S658!frank\nFrom: frank@D012S658.uucp (Frank O'Dwyer)\nNewsgroups: talk.abortion,alt.atheism,talk.religion.misc\nSubject: Re: After 2000 years, can we say that Christian Morality is\nDate: 15 Apr 1993 21:12:01 GMT\nOrganization: Siemens-Nixdorf AG\nLines: 50\nMessage-ID: <1qkj31$4c6@horus.ap.mchp.sni.de>\nReferences: <pww-140493214334@spac-at1-59.rice.edu> <1qjahh$mrs@horus.ap.mchp.sni.de> <lsr6ihINNsa@exodus.Eng.Sun.COM>\nNNTP-Posting-Host: d012s658.ap.mchp.sni.de",
   'In article <lsr6ihINNsa@exodus.Eng.Sun.COM> emarsh@hernes-sun.Eng.Sun.COM (Eric Marsh) writes:\n#In article <1qjahh$mrs@horus.ap.mchp.sni.de> frank@D012S658.uucp (Frank O\'Dwyer) writes:\n#>Science ("the real world") has its basis in va

In [26]:
def extraction_informations_entete(entete):
  l=entete.split("\n")
  d=dict()
  for x in l:
    z=x.split(":")
    if len(z)==2:
       d[z[0]]=z[1]

  return d

In [27]:
# extraire l'organisation du premier document du rdd1 (alt.atheism)
# par l'action first
doc1=rdd1.first()
entete=doc1[1][0]
# Dictionnaire des informations de l'entete
dict1_info=extraction_informations_entete(entete)
dict1_info

{'Path': ' cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!uunet!mcsun!Germany.EU.net!thoth.mchp.sni.de!horus.ap.mchp.sni.de!D012S658!frank',
 'From': " frank@D012S658.uucp (Frank O'Dwyer)",
 'Newsgroups': ' talk.abortion,alt.atheism,talk.religion.misc',
 'Organization': ' Siemens-Nixdorf AG',
 'Lines': ' 50',
 'Message-ID': ' <1qkj31$4c6@horus.ap.mchp.sni.de>',
 'References': ' <pww-140493214334@spac-at1-59.rice.edu> <1qjahh$mrs@horus.ap.mchp.sni.de> <lsr6ihINNsa@exodus.Eng.Sun.COM>',
 'NNTP-Posting-Host': ' d012s658.ap.mchp.sni.de'}

In [28]:
# selectionner Organization et Newsgroups
print(dict1_info['Newsgroups'])
print(dict1_info['Organization'])

 talk.abortion,alt.atheism,talk.religion.misc
 Siemens-Nixdorf AG


In [29]:
# par l'action take
doc2=rdd2.take(2)
entete2=doc2[1][1][0]
# Dictionnaire des informations de l'entete
dict2_info=extraction_informations_entete(entete2)
dict2_info

{'Path': ' cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!zaphod.mps.ohio-state.edu!hobbes.physics.uiowa.edu!news.iastate.edu!iscsvax.uni.edu!reed5575',
 'From': ' reed5575@iscsvax.uni.edu',
 'Newsgroups': ' rec.sport.baseball',
 'Message-ID': ' <1993Apr20.233429.12558@iscsvax.uni.edu>',
 'References': ' <1993Apr20.102857.1@tesla.njit.edu>',
 'Organization': ' University of Northern Iowa',
 'Lines': ' 12'}

In [30]:
# selectionner Organization et Newsgroups
print(dict2_info['Newsgroups'])
print(dict2_info['Organization'])
#print(dict2_info['Subject'])
print(dict2_info['References'])

 rec.sport.baseball
 University of Northern Iowa
 <1993Apr20.102857.1@tesla.njit.edu>


In [31]:
def extraction_informations_entete2(entete):
  if entete is not None:
    l=entete[1][0].split("\n")
    d=dict()
    if len(l)!=0:
      for x in l:
        z=x.split(":",1)
        if len(z)==2:
          d[z[0]]=z[1]
        if len(z)<2:
           pass
        

      return (entete[0],[d,entete[1][1]])

In [32]:
new_rdd1=rdd1.map(extraction_informations_entete2)
#rdd1.take(3)

In [34]:
new_rdd1.take(8)

[('file:/content/20_newsgroups/alt.atheism/53079',
  [{'Xref': ' cantaloupe.srv.cs.cmu.edu talk.abortion:120533 alt.atheism:53079 talk.religion.misc:83551',
    'Path': ' cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!uunet!mcsun!Germany.EU.net!thoth.mchp.sni.de!horus.ap.mchp.sni.de!D012S658!frank',
    'From': " frank@D012S658.uucp (Frank O'Dwyer)",
    'Newsgroups': ' talk.abortion,alt.atheism,talk.religion.misc',
    'Subject': ' Re: After 2000 years, can we say that Christian Morality is',
    'Date': ' 15 Apr 1993 21:12:01 GMT',
    'Organization': ' Siemens-Nixdorf AG',
    'Lines': ' 50',
    'Message-ID': ' <1qkj31$4c6@horus.ap.mchp.sni.de>',
    'References': ' <pww-140493214334@spac-at1-59.rice.edu> <1qjahh$mrs@horus.ap.mchp.sni.de> <lsr6ihINNsa@exodus.Eng.Sun.COM>',
    'NNTP-Posting-Host': ' d012s658.ap.mchp.sni.de'},
   'In article <lsr6ihINNsa@exodus.Eng.Sun.COM> emarsh@hernes-sun.Eng.Sun.COM (Eric Marsh) writes:\n#In article <1qjahh$mrs@horus.ap.mchp.sni.de>

In [33]:
new_rdd2=rdd2.map(extraction_informations_entete2)

In [35]:
new_rdd2.take(8)

[('file:/content/20_newsgroups/rec.sport.baseball/104336',
  [{'Newsgroups': ' rec.sport.baseball',
    'Path': ' cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!usenet.ins.cwru.edu!gatech!purdue!mentor.cc.purdue.edu!mace.cc.purdue.edu!poutsmaj',
    'From': ' poutsmaj@mace.cc.purdue.edu (unknown)',
    'Subject': ' Re: Aguilera Causes Cardiac Arrest',
    'Message-ID': ' <C5J68F.FC@mentor.cc.purdue.edu>',
    'Sender': ' news@mentor.cc.purdue.edu (USENET News)',
    'Organization': ' Purdue University',
    'References': ' <1993Apr13.172643.15442@sctc.com> <1993Apr14.123722.24506@bmw.mayo.edu> <1qi97dINNemh@phakt.usc.edu>',
    'Date': ' Thu, 15 Apr 1993 15:12:14 GMT',
    'Lines': ' 21'},
   "In article <1qi97dINNemh@phakt.usc.edu> wagner@phakt.usc.edu (Loren Wagner) writes:\n>In article <1993Apr14.123722.24506@bmw.mayo.edu> bergerson@mayo.edu writes:\n>>\n>>Maybe you would rather have Ron Davis back ????   :^)\n>>\n>\n>The truly amazing thing was h

In [91]:
# Fusionner les deux RDD (union)
fusion=new_rdd1.union(new_rdd2)


In [93]:
doc1=fusion.take(4)
doc1


[('file:/content/20_newsgroups/alt.atheism/53079',
  [{'Xref': ' cantaloupe.srv.cs.cmu.edu talk.abortion:120533 alt.atheism:53079 talk.religion.misc:83551',
    'Path': ' cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!uunet!mcsun!Germany.EU.net!thoth.mchp.sni.de!horus.ap.mchp.sni.de!D012S658!frank',
    'From': " frank@D012S658.uucp (Frank O'Dwyer)",
    'Newsgroups': ' talk.abortion,alt.atheism,talk.religion.misc',
    'Subject': ' Re: After 2000 years, can we say that Christian Morality is',
    'Date': ' 15 Apr 1993 21:12:01 GMT',
    'Organization': ' Siemens-Nixdorf AG',
    'Lines': ' 50',
    'Message-ID': ' <1qkj31$4c6@horus.ap.mchp.sni.de>',
    'References': ' <pww-140493214334@spac-at1-59.rice.edu> <1qjahh$mrs@horus.ap.mchp.sni.de> <lsr6ihINNsa@exodus.Eng.Sun.COM>',
    'NNTP-Posting-Host': ' d012s658.ap.mchp.sni.de'},
   'In article <lsr6ihINNsa@exodus.Eng.Sun.COM> emarsh@hernes-sun.Eng.Sun.COM (Eric Marsh) writes:\n#In article <1qjahh$mrs@horus.ap.mchp.sni.de>

In [54]:
fusion.take(30)

[('file:/content/20_newsgroups/alt.atheism/53079',
  [{'Xref': ' cantaloupe.srv.cs.cmu.edu talk.abortion:120533 alt.atheism:53079 talk.religion.misc:83551',
    'Path': ' cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!uunet!mcsun!Germany.EU.net!thoth.mchp.sni.de!horus.ap.mchp.sni.de!D012S658!frank',
    'From': " frank@D012S658.uucp (Frank O'Dwyer)",
    'Newsgroups': ' talk.abortion,alt.atheism,talk.religion.misc',
    'Subject': ' Re: After 2000 years, can we say that Christian Morality is',
    'Date': ' 15 Apr 1993 21:12:01 GMT',
    'Organization': ' Siemens-Nixdorf AG',
    'Lines': ' 50',
    'Message-ID': ' <1qkj31$4c6@horus.ap.mchp.sni.de>',
    'References': ' <pww-140493214334@spac-at1-59.rice.edu> <1qjahh$mrs@horus.ap.mchp.sni.de> <lsr6ihINNsa@exodus.Eng.Sun.COM>',
    'NNTP-Posting-Host': ' d012s658.ap.mchp.sni.de'},
   'In article <lsr6ihINNsa@exodus.Eng.Sun.COM> emarsh@hernes-sun.Eng.Sun.COM (Eric Marsh) writes:\n#In article <1qjahh$mrs@horus.ap.mchp.sni.de>

In [175]:
from pyspark.sql import Row
def ToRow(x):
  entete=x[1][0]
  
    #dict1_info=extraction_informations_entete(entete)
  row = Row(Path=x[0], Newsgroups=entete['Newsgroups'] if 'Newsgroups' in entete.keys() else None,
              Organization=entete['Organization'] if 'Organization' in entete.keys() else None,
              Subject=entete['Subject'] if 'Subject' in entete.keys() else None,
              Date=entete['Date'] if 'Date' in entete.keys() else None,
            Lines=entete['Lines'] if 'Lines' in entete.keys() else None,
            From=entete['From'] if 'From' in entete.keys() else None,
              
              Contenu=x[1][1])
  

  return row

#fusion.take(1)
rddF=fusion.map(ToRow)


In [176]:
rddF.take(60)

[Row(Path='file:/content/20_newsgroups/alt.atheism/53079', Newsgroups=' talk.abortion,alt.atheism,talk.religion.misc', Organization=' Siemens-Nixdorf AG', Subject=' Re: After 2000 years, can we say that Christian Morality is', Date=' 15 Apr 1993 21:12:01 GMT', Lines=' 50', From=" frank@D012S658.uucp (Frank O'Dwyer)", Contenu='In article <lsr6ihINNsa@exodus.Eng.Sun.COM> emarsh@hernes-sun.Eng.Sun.COM (Eric Marsh) writes:\n#In article <1qjahh$mrs@horus.ap.mchp.sni.de> frank@D012S658.uucp (Frank O\'Dwyer) writes:\n#>Science ("the real world") has its basis in values, not the other way round, \n#>as you would wish it.  If there is no such thing as objective value, then \n#>science can not objectively be said to be more useful than a kick in the head.\n#>Simple theories with accurate predictions could not objectively be said\n#>to be more useful than a set of tarot cards.  You like those conclusions?\n#>I don\'t.\n#\n#I think that you are changing the meaning of "values" here. Perhaps\n#it i

In [177]:
#Create dataframe
df=spark.createDataFrame(rddF)
df.printSchema()
df.show()
df.count()

root
 |-- Path: string (nullable = true)
 |-- Newsgroups: string (nullable = true)
 |-- Organization: string (nullable = true)
 |-- Subject: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Lines: string (nullable = true)
 |-- From: string (nullable = true)
 |-- Contenu: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
|                Path|          Newsgroups|        Organization|             Subject|                Date|Lines|                From|             Contenu|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
|file:/content/20_...| talk.abortion,al...|  Siemens-Nixdorf AG| Re: After 2000 y...| 15 Apr 1993 21:1...|   50| frank@D012S658.u...|In article <lsr6i...|
|file:/content/20_...|         alt.atheism| Bell-Northern Re...| 

2000

In [67]:
!pip install pandavro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 71.9 MB/s eta 0:00:00
  Created wheel for pandavro: filename=pandavro-1.7.1-py3-none-any.whl size=5688 sha256=be2261f1b3392d667f68a2f253bf693a690dd1afcd25c64ebfcd3160f93d209a
  Stored in directory: /root/.cache/pip/wheels/85/cc/15/480a3cfa1a9fc49e2af9bde7437bd6c8c0265f17e61f32672b
Successfully built pandavro


In [178]:
import numpy as np
import pandas as pd
import pandavro as pdx
filename = "df.avro"
df2=df.toPandas()
pdx.to_avro(filename, df2)

In [179]:
#lecture du fichier avro créé
saved = pdx.read_avro(filename)
print(saved)

                                                   Path  \
0         file:/content/20_newsgroups/alt.atheism/53079   
1         file:/content/20_newsgroups/alt.atheism/51141   
2         file:/content/20_newsgroups/alt.atheism/53282   
3         file:/content/20_newsgroups/alt.atheism/53631   
4         file:/content/20_newsgroups/alt.atheism/53163   
...                                                 ...   
1995  file:/content/20_newsgroups/rec.sport.baseball...   
1996  file:/content/20_newsgroups/rec.sport.baseball...   
1997  file:/content/20_newsgroups/rec.sport.baseball...   
1998  file:/content/20_newsgroups/rec.sport.baseball...   
1999  file:/content/20_newsgroups/rec.sport.baseball...   

                                         Newsgroups  \
0      talk.abortion,alt.atheism,talk.religion.misc   
1                                       alt.atheism   
2                                       alt.atheism   
3                                       alt.atheism   
4               

In [180]:
# sauvgarder le dataframe au format parquet
import pyarrow as pa
import pyarrow.parquet as pq
table = pa.Table.from_pandas(df2)
pq.write_table(table, 'df.parquet')

In [182]:
# lecture du df a partir du fichier parquet
table2 = pq.read_table('df.parquet')
table2.to_pandas()

,Path,Newsgroups,Organization,Subject,Date,Lines,From,Contenu
0,file:/content/20_newsgroups/alt.atheism/53079,"talk.abortion,alt.atheism,talk.religion.misc",Siemens-Nixdorf AG,"Re: After 2000 years, can we say that Christi...",15 Apr 1993 21:12:01 GMT,50,frank@D012S658.uucp (Frank O'Dwyer),In article <lsr6ihINNsa@exodus.Eng.Sun.COM> em...
1,file:/content/20_newsgroups/alt.atheism/51141,alt.atheism,"Bell-Northern Research, Ottawa, Canada",Re: Speculations,5 Apr 93 21:19:24 GMT,17,dgraham@bmers30.bnr.ca (Douglas Graham),In article <930405.172903.4w6.rusnews.w165w@ma...
2,file:/content/20_newsgroups/alt.atheism/53282,alt.atheism,Orden del Lobo Estepario,Studies on Book of Mormon,"Sun, 18 Apr 1993 14:15:33 CST",20,agrino@enkidu.mic.cl (Andres Grino Brandt),Hi!
3,file:/content/20_newsgroups/alt.atheism/53631,alt.atheism,Nottingham University,Re: Death Penalty (was Re: Political Atheists?),"Wed, 21 Apr 93 11:49:47 GMT",60,eczcaw@mips.nott.ac.uk (A.Wainwright),In article <C5rLyz.4Mt@darkside.osrhe.uoknor.e...
4,file:/content/20_newsgroups/alt.atheism/53163,alt.atheism,Boston University Physics Department,Re: The Inimitable Rushdie,16 Apr 93 01:31:01 GMT,41,jaeger@buphy.bu.edu (Gregg Jaeger),In article <C5HKv2.Epv@blaze.cs.jhu.edu> arrom...
...,...,...,...,...,...,...,...,...
1995,file:/content/20_newsgroups/rec.sport.baseball...,rec.sport.baseball,the HP Corporate notes server,Rockies 5 - 8,"Thu, 22 Apr 1993 15:08:01 GMT",37,boell@hpcc01.corp.hp.com (Donald P Boell),\nName Pos AB H 2B 3B ...
1996,file:/content/20_newsgroups/rec.sport.baseball...,rec.sport.baseball,"Ball State University, Muncie, In - Univ. Com...",Jim Lefebvre is an idiot.,6 Apr 93 00:01:41 GMT,14,00bjgood@leo.bsuvc.bsu.edu,I just wanted to let everyone know that I have...
1997,file:/content/20_newsgroups/rec.sport.baseball...,rec.sport.baseball,"Computing Services Division, University of Wi...",Re: Twins Update 4-22,23 Apr 1993 17:05:48 GMT,12,paul@csd4.csd.uwm.edu (Paul R Krueger),In article <1993Apr23.151050.8995@sctc.com> ma...
1998,file:/content/20_newsgroups/rec.sport.baseball...,rec.sport.baseball,Brandeis University,Re: Best Sportwriters...,"Fri, 16 Apr 1993 07:22:59 GMT",23,st902415@pip.cc.brandeis.edu (Adam Levin),"In article <C5K7nK.7tv@news.cso.uiuc.edu>, rko..."


In [189]:
df.groupBy("Newsgroups").count().show()

+--------------------+-----+
|          Newsgroups|count|
+--------------------+-----+
| talk.religion.mi...|    2|
| sci.skeptic,alt....|    7|
| alt.atheism,talk...|    3|
| alt.atheism,rec....|    3|
| alt.atheism,talk...|   92|
| soc.culture.arab...|    2|
| alt.atheism,talk...|    5|
| alt.atheism,alt....|    2|
| alt.drugs,alt.at...|    2|
| alt.atheism,talk...|    2|
| alt.atheism,soc....|    8|
|         alt.atheism|  734|
| alt.atheism,alt....|    7|
| talk.religion.mi...|    4|
| alt.atheism,talk...|   20|
| alt.slack,talk.r...|    1|
| talk.religion.mi...|    4|
| talk.abortion,al...|   94|
| alt.atheism,soc....|    1|
| talk.abortion,al...|    1|
+--------------------+-----+
only showing top 20 rows



In [190]:
spark.sql("DROP VIEW sport")

AnalysisException: ignored

In [191]:
spark.sql("DROP VIEW religion")

DataFrame[]

In [192]:
from pandas._libs.hashtable import value_count
df.createOrReplaceTempView("Document")
#sqlDF = spark.sql("SELECT count(*) FROM Document WHERE Newsgroups like '%rec.sport.baseball%' ")
#sqlDF = spark.sql("select count(*) from Document WHERE ( Newsgroups LIKE '%alt.atheism%' and Newsgroups Like '%rec.sport.baseball%')")
#sqlDF = spark.sql("CREATE TEMP VIEW sport AS SELECT Newsgroups FROM Document WHERE Newsgroups LIKE '%rec.sport.baseball%'")
#sqlDF.show()







In [195]:
sqlDF = spark.sql("CREATE TEMP VIEW sport AS SELECT Newsgroups FROM Document WHERE Newsgroups LIKE '%rec.sport.baseball%'")
sqlDF.show()

++
||
++
++



In [196]:
sqlDF = spark.sql("CREATE TEMP VIEW religion AS SELECT Newsgroups FROM Document WHERE Newsgroups LIKE '%alt.atheism%'")
sqlDF.show()

AnalysisException: ignored

In [197]:
sqlDF2 = spark.sql("SELECT * FROM sport")
sqlDF2.show()
sqlDF3 = spark.sql("SELECT * FROM religion")
sqlDF3.show()

+-------------------+
|         Newsgroups|
+-------------------+
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
| rec.sport.baseball|
+-------------------+
only showing top 20 rows

+--------------------+
|          Newsgroups|
+--------------------+
| talk.abortion,al...|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
| talk.abortion,al...|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
| alt.atheism,talk...|
|         alt.atheism|
|         alt.atheism|
|        

In [198]:
# c pour prouver que ya pas d'intersection entre les deux catégories 
# de plus on a 1000 lignes dans la vue religion et 1000 dans la vue religion
#(en sachant que le nombre total des lignes dans la table document est de 2000)
sqlDF3 = spark.sql("SELECT * FROM religion INTERSECT (SELECT * FROM sport) ")
sqlDF3.show()

+----------+
|Newsgroups|
+----------+
+----------+



In [199]:
# le nombre de documents dont la categorie contient rec_baseball
sqlDF3 = spark.sql("SELECT count(s.Newsgroups) as nbr_rec_baseball FROM sport s ")
sqlDF3.show()

+----------------+
|nbr_rec_baseball|
+----------------+
|            1000|
+----------------+



In [200]:
# le nombre de documents dont la categorie contient alt.atheism
sqlDF3 = spark.sql("SELECT count(Newsgroups) as nb_alt_atheism FROM religion")
sqlDF3.show()
 

+--------------+
|nb_alt_atheism|
+--------------+
|          1000|
+--------------+



In [201]:
#le nombre d'organisations différentes
from pyspark.sql.functions import countDistinct
df2=df.select(countDistinct("Organization"))
df2.show()

+----------------------------+
|count(DISTINCT Organization)|
+----------------------------+
|                         485|
+----------------------------+



In [202]:
# Max, Min , Moyenne par rapport a l'attribut Lines
df.describe("Lines").show()

+-------+------------------+
|summary|             Lines|
+-------+------------------+
|  count|              1994|
|   mean| 36.48294884653962|
| stddev|48.791372734500946|
|    min|                 1|
|    max|                99|
+-------+------------------+



In [204]:
# le nombre de documents par organization
df.groupBy("Organization").count().show()

+--------------------+-----+
|        Organization|count|
+--------------------+-----+
|           SunSelect|    2|
| Case Western Res...|   39|
| University of Br...|    1|
| Princeton Univer...|   24|
| University of Ne...|    2|
| S-CUBED, A Divis...|    1|
| Harlequin Ltd. C...|    1|
| IBM T.J. Watson ...|    1|
| University of Ne...|    7|
|   Cured, discharged|    3|
| Penn State Engin...|    1|
|   Augustana College|    1|
|        SGI TechPubs|    1|
| National Optical...|    2|
| University of Ce...|    1|
| Cabletron System...|    6|
| Red Wolfe @ The ...|    1|
| Welch Medical Li...|    2|
| Johns Hopkins Un...|    7|
|  Indiana University|   17|
+--------------------+-----+
only showing top 20 rows



In [205]:
# le nombre de documents qui ont été créés pour chaque auteur
df.groupBy("FROM").count().show()

+--------------------+-----+
|                FROM|count|
+--------------------+-----+
| kv07@IASTATE.EDU...|    1|
| dpw@sei.cmu.edu ...|    4|
| mccullou@snake2....|    6|
| markp@avignon (M...|    1|
| livesey@solntze....|   70|
| jvigneau@cs.ulow...|    1|
| kilman2y@fiu.edu...|    1|
| darice@yoyo.cc.m...|   14|
| kax@cs.nott.ac.u...|    2|
| acooper@mac.cc.m...|    4|
| pzimmerm@mail.sa...|    1|
| ed@wente.llnl.go...|    1|
| halat@pooh.bears...|   15|
| magney@cco.calte...|    1|
| dewey@risc.sps.m...|    3|
| mikec@sail.LABS....|    2|
| sbradley@scic.in...|    2|
| adpeters@sunflow...|    3|
| tgk@cs.toronto.e...|    2|
| simon@dcs.warwic...|    3|
+--------------------+-----+
only showing top 20 rows

